In [67]:
import os

from astroquery.vizier import Vizier
from astroquery.simbad import Simbad

skipDirs = ['.git', 'general']

In [69]:
simbadGaiaIds = {}

for objName in os.listdir(os.getcwd()):
	if os.path.isdir(objName) and objName not in skipDirs:
		simbadResult = Simbad.query_objectids(objName)
		allIds = simbadResult['ID']
		for astroId in allIds:
			if 'Gaia DR3' in astroId:
				simbadGaiaIds[objName] = float(astroId.replace('Gaia DR3 ', ''))
simbadGaiaIds

{'LINEAR_44914': 9.342673008441536e+17,
 'LINEAR_16694484': 1.3849724758727135e+18,
 'TYC4002-2628-1': 1.996670926588668e+18,
 'CRTS J074502.6+253437': 8.687838929873417e+17,
 'ATO J172.8778+47.9056': 7.88873415117892e+17}

In [76]:
gaiaSourceInfo = {}

for objName in os.listdir(os.getcwd()):
	if os.path.isdir(objName) and objName not in skipDirs:
		tableListResult = Vizier.query_object(objName, catalog='I/355/gaiadr3') # query Gaia DR3 table
		gaiaSourceInfo[objName] = tableListResult[0]
		print(objName)
		print(tableListResult[0]['Source'])

LINEAR_44914
      Source      
------------------
934266403195100160
934266506274312960
934266544928840704
934266957246768512
934267227828824960
934267232124677760
934267232124677888
934267262190928384
934267300844153600
934267330908047360
934267369563630720
934267537066481408
934267571426220672
934267674505434496
934267678801274752
934267713161012992
934267743224911872
934267747520751872
LINEAR_16694484
       Source      
-------------------
1384971440784499584
1384971479440297472
1384971651238990464
1384971715662586880
1384971715664168576
1384971719958467968
1384971719958640896
1384971754317308928
1384972059259805952
1384972402857181952
1384972402857182080
1384972407153685632
1384972437216924288
1384972437216927616
1384972471576664960
1384972475872713472
1384972475872713984
1384972505940223744
1384972574655887744
1384972578951929216
1384972613311668736
1384972746454578816
1384972780814324864
TYC4002-2628-1
       Source      
-------------------
1996669891491318272
1996669891491322

In [43]:
gaiaTable = list(gaiaSourceInfo.values())[0]
gaiaTable.info

<Table length=18>
   name    dtype       unit       format                                                           description                                                           n_bad
--------- ------- ------------- --------- ------------------------------------------------------------------------------------------------------------------------------ -----
  RA_ICRS float64           deg {:15.11f}                                                                                       Right ascension (ICRS) at Ep=2016.0 (ra)     0
  DE_ICRS float64           deg {:15.11f}                                                                                          Declination (ICRS) at Ep=2016.0 (dec)     0
   Source   int64                                                                         Unique source identifier (unique within a particular Data Release) (source_id)     0
e_RA_ICRS float64           mas   {:7.4f}                                                                  

In [66]:
class HRSourceMags:

	def __init__(self, obj_name, gdr3_source_id, g_mag, bp_rp) -> None:
		self.obj_name = obj_name
		self.gdr3_source_id = gdr3_source_id
		self.g_mag = g_mag
		self.bp_rp = bp_rp

	def __str__(self) -> str:
		return f"{self.obj_name}|{self.gdr3_source_id}: G={self.g_mag}, BP-RP={self.bp_rp}"
	
	def __repr__(self) -> str:
		return self.__str__()

objColors: list[HRSourceMags] = []

debugTable: None
for obj, gaiaTable in gaiaSourceInfo.items():
	print(obj, simbadGaiaIds[obj])
	matchedSourceInfo = gaiaTable[gaiaTable['Source'] == float(simbadGaiaIds[obj])]
	try:
		objColors.append(HRSourceMags(obj, float(simbadGaiaIds[obj]), matchedSourceInfo['Gmag'][0], matchedSourceInfo['BP-RP'][0]))
	except Exception as e:
		print('\tERR')
		debugTable = matchedSourceInfo
objColors
debugTable

LINEAR_44914 934267300844153600
LINEAR_16694484 1384972475872713472
TYC4002-2628-1 1996670926588667904
index 0 is out of bounds for axis 0 with size 0
CRTS J074502.6+253437 868783892987341696
ATO J172.8778+47.9056 788873415117891968


RA_ICRS,DE_ICRS,Source,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RUWE,FG,e_FG,Gmag,FBP,e_FBP,BPmag,FRP,e_FRP,RPmag,BP-RP,RV,e_RV,Vbroad,GRVSmag,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,Teff,logg,__Fe_H_,Dist,A0,HIP,PS1,SDSS13,SKYM2,TYC2,URAT1,AllWISE,APASS9,GSC23,RAVE5,_2MASS,RAVE6,RAJ2000,DEJ2000
deg,deg,,mas,mas,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,mag,,,mag,,,mag,mag,km / s,km / s,km / s,mag,,,,,,,,,,,,K,log(cm.s**-2),,pc,mag,,,,,,,,,,,,,deg,deg
float64,float64,int64,float64,float64,float64,float32,float64,float64,float32,float64,float32,float64,float64,float32,float64,float32,float32,float64,float32,float32,float64,float64,float64,float32,float64,float64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,float64,float64,float64,float64,float64,int32,int64,int64,int32,str12,str15,str19,int32,str10,str16,str17,str21,float64,float64
